In [4]:
import numpy as np
import argparse
import time
import cv2
import os
import cv2
import glob
import face_recognition

# construct the argument parse and parse the arguments
# ap = argparse.ArgumentParser()
# ap.add_argument("-i", "--image", required=True,
#     help="path to input image")
# ap.add_argument("-c", "--confidence", type=float, default=0.5,
#     help="minimum probability to filter weak detections, IoU threshold")
# ap.add_argument("-t", "--threshold", type=float, default=0.3,
#     help="threshold when applying non-maxima suppression")
# args = vars(ap.parse_args())

default_confidence = 0.5
default_threshold = 0.4

labelsPath = 'yolo/coco.names'
weightsPath = 'yolo/yolov3-spp.weights'
configPath = 'yolo/yolov3-spp.cfg'
i = 0
path = 'blacklist-captured/'
# read class names from text file 
LABELS = open(labelsPath).read().strip().split("\n")

# read blacklist names from text file 
BLACKLIST = open('blacklist.names').read().strip().split("\n")

# generate different colors to represent each label
COLORS = np.random.uniform(0, 255, size=(len(LABELS), 3))

# read the pre trained model on config file
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

# get output layer
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

known_face_encodings = []
known_face_names = []

frame_resizing = 0.25

def load_encoding_images(images_path):
       
    images_path = glob.glob(os.path.join(images_path, "*.*"))

    print("{} encoding images found.".format(len(images_path)))

        
    for img_path in images_path:
        img = cv2.imread(img_path)
        rgb_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

           
        basename = os.path.basename(img_path)
        (filename, ext) = os.path.splitext(basename)
           
        img_encoding = face_recognition.face_encodings(rgb_img)[0]

            
        known_face_encodings.append(img_encoding)
        known_face_names.append(filename)
    print("Encoding images loaded")

def detect_known_faces(frame):
    small_frame = cv2.resize(frame, (0, 0), fx=frame_resizing, fy=frame_resizing)
    rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)
    face_locations = face_recognition.face_locations(rgb_small_frame)
    face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

    face_names = []
    for face_encoding in face_encodings:
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = "Unknown"

        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
        best_match_index = np.argmin(face_distances)
        if matches[best_match_index]:
            name = known_face_names[best_match_index]
        face_names.append(name)

        face_locations = np.array(face_locations)
        face_locations = face_locations / frame_resizing
        return face_locations.astype(int), face_names

load_encoding_images("faces/")

def detect_face_in_person(frame):
    # print(path)
    try:
        face_locations, face_names = detect_known_faces(frame)
        for face_loc, name in zip(face_locations, face_names):
            y1, x2, y2, x1 = face_loc[0], face_loc[1], face_loc[2], face_loc[3]
            cv2.putText(frame, name,(x1, y1 - 10), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 200), 2)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 200), 4)
            if name in BLACKLIST:
                global i
                filename = f'{path}{name}{i}.jpg'
                cv2.imwrite("blacklist-captured/%s%d.jpg" % (name, i), frame)
                i+=1
                # print(path)
    except:
        pass
    return frame


def draw_bounding_box(image, classID, confidence, x, y, w, h):
    color = COLORS[classID]
    cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
    text = "{}: {:.4f}".format(LABELS[classID], confidence)
    cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,
        0.5, color, 2)
    
def object_detection_in_image(image):
    #create input blob - bgr to rgb
    blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (320, 320),
        swapRB=True, crop=False)
    net.setInput(blob)
    layerOutputs = net.forward(output_layers)
    
    boxes = []
    confidences = []
    classIDs = []

    for output in layerOutputs:
        # loop over each of the detections
        for detection in output:
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]

            # ignore weak predictions
            if confidence > default_confidence:
                # scale the bounding box coordinates back relative to the size of the image
                box = detection[0:4] * np.array([W, H, W, H])
                (centerX, centerY, width, height) = box.astype("int")

                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))

                boxes.append([x, y, int(width), int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)

    # apply non-maxima suppression
    idices = cv2.dnn.NMSBoxes(boxes, confidences, default_confidence,
        default_threshold)

    if len(idices) > 0:
        for i in idices.flatten():
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            if(classIDs[i]==0):
                image = detect_face_in_person(image)
                draw_bounding_box(image, classIDs[i], confidences[i], x, y, w, h)
            else:
                draw_bounding_box(image, classIDs[i], confidences[i], x, y, w, h)
            


# reading the input frames from the video
cap = cv2.VideoCapture(0)

while 1:
    # reads frames from a camera
    ret, image = cap.read()
    (H, W) = image.shape[:2]
    
    object_detection_in_image(image)
    cv2.imshow('imgage', image)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
#release the object
cap.release()
  
# Close the window
cv2.destroyAllWindows()

2 encoding images found.
Encoding images loaded
